In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [15]:
#Get the coordinates while specifying column names
filename = 'https://cocl.us/Geospatial_data'
Lat_long_coord_df = pd.read_csv(filename)
Lat_long_coord_df.columns = ["Postal_Code","Latitude","Longitude"]


In [11]:
#create Toronto data file from part 1 
from bs4 import BeautifulSoup
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# Create a BeautifulSoup object
soup = BeautifulSoup(page.text, 'html.parser')
# Create a data frame from BeautifulSoup object soup
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
Toronto_df=df

# Drop whole row with 'Not assigned' in "Borough" column
Toronto_df1=Toronto_df[Toronto_df.Borough != 'Not assigned']

# reset index, because we droped two rows
Toronto_df1.reset_index(drop=True, inplace=True)

#Replace Not assigned value with Borough for Neighbourhood
Toronto_df1.Neighbourhood.replace("Not assigned",Toronto_df1.Borough,inplace=True)

#More than one neighborhood can exist in one postal code area. 
#For example, in the table on the Wikipedia page, 
#you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. 
#These two rows will be combined into one row with the neighborhoods separated with a comma

Toronto_df2 = Toronto_df1.groupby(by=['Postcode','Borough']).agg(lambda x: ','.join(x))
#Reset index to Postcode and Borough
Toronto_df2.reset_index(level=['Postcode','Borough'], inplace=True)

#rename columns 
Toronto_df2.rename(columns={'Postcode': 'PostalCode', 'Neighbourhood': 'Neighborhood'}, inplace=True)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [12]:
Toronto_df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
Lat_long_coord_df.head()

,Latitude,Longitude
Postal_Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [16]:
Toronto_merged = Toronto_df2

# merge Toronto_merged with Lat_long_coord_df to add latitude/longitude for each postal code
Toronto_merged = Toronto_merged.join(Lat_long_coord_df.set_index('Postal_Code'), on='PostalCode')

Toronto_merged.head() # check all columns

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
